## 07: Interactive Facility Scenario Explorer

**Goal:** To create a simple, user-driven tool for exploring "what-if" scenarios in facility planning. Static maps are useful, but interactive tools allow stakeholders to get immediate feedback on their ideas.

**Methodology:**
1.  Use `ipywidgets` and `ipyleaflet` to create an interactive map directly within Jupyter.
2.  Display existing facilities and a set of demand points.
3.  Allow a user to **click on the map to propose a new facility location**.
4.  Dynamically recalculate which demand points would be served by this new facility and update the map instantly.

### 1. Setup and Library Imports

**Important:** If you haven't already, you will need to install these libraries and enable the notebook extension. Run these commands in your terminal (with your `spatial-ml-env` activated), and then **restart the Jupyter kernel**.

```bash
pip install ipywidgets ipyleaflet
jupyter nbextension enable --py widgetsnbextension
```

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from ipyleaflet import Map, Marker, GeoData, basemaps
import ipywidgets as widgets
from IPython.display import display
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2. Create Synthetic Data

We'll generate a set of existing facilities and a grid of demand points representing residential areas.

In [2]:
np.random.seed(123)

# Existing facilities that will be present at the start
existing_facilities = gpd.GeoDataFrame(
    {'name': ['Existing Site A', 'Existing Site B']},
    geometry=[Point(-1.85, 51.56), Point(-1.75, 51.58)],
    crs="EPSG:4326"
)

# A grid of demand points
xx, yy = np.meshgrid(np.linspace(-1.9, -1.7, 10), np.linspace(51.55, 51.6, 10))
demand_points = gpd.GeoDataFrame(
    geometry=[Point(x, y) for x, y in zip(xx.ravel(), yy.ravel())],
    crs="EPSG:4326"
)
demand_points['id'] = range(len(demand_points))

# This dataframe will hold the current state (existing + proposed sites)
# We reset it to a copy of the original facilities when the reset button is pressed.
all_facilities = existing_facilities.copy()

### 3. Core Logic: Allocate Demand and Create Map Layers

This function takes the current set of facilities and demand points, calculates allocations, and prepares the data for display on the interactive map.

In [3]:
def allocate_demand_and_get_layers(facilities_gdf, demand_gdf):
    """Calculates which facility serves each demand point and prepares layers for the map."""
    # If there are no facilities, color all demand points as unserved
    if facilities_gdf.empty:
        demand_gdf['color'] = 'gray'
        demand_layer = GeoData(geo_dataframe=demand_gdf, 
                               point_style={'radius': 5, 'color': 'gray', 'weight': 1, 'fillOpacity': 0.7},
                               name='Unserved Demand')
        return [demand_layer], []

    # Project to a meter-based CRS for accurate distance calculation
    facilities_proj = facilities_gdf.to_crs(epsg=32630)
    demand_proj = demand_gdf.to_crs(epsg=32630)

    # For each demand point, find the nearest facility
    allocations = []
    for idx, demand_point in demand_proj.iterrows():
        distances = facilities_proj.distance(demand_point.geometry)
        closest_idx = distances.idxmin()
        allocations.append(closest_idx)
    
    demand_proj['allocated_to_idx'] = allocations

    # Create a consistent color map for the facilities
    unique_facility_indices = sorted(demand_proj['allocated_to_idx'].unique())
    color_map = {fac_idx: colors.to_hex(cm.tab10(i)) for i, fac_idx in enumerate(unique_facility_indices)}
    demand_proj['color'] = demand_proj['allocated_to_idx'].map(color_map)

    # Convert back to WGS84 for ipyleaflet display
    demand_final = demand_proj.to_crs(epsg=4326)

    # Create map layers
    demand_layer = GeoData(geo_dataframe=demand_final, 
                           point_style={'radius': 5, 'weight': 0.5},
                           style_callback=lambda feat: {'fillColor': feat['properties']['color'], 'color': feat['properties']['color'], 'fillOpacity': 0.8},
                           name='Demand')
    
    facility_markers = [Marker(location=(g.y, g.x), draggable=False, title=name) 
                        for g, name in zip(facilities_gdf.geometry, facilities_gdf.name)]
    
    return [demand_layer], facility_markers

### 4. Build the Interactive Map

This is where we tie everything together. We define functions to handle map updates and user clicks, then wire them up to the map and button widgets.

In [4]:
# Initialize map centered on our area of interest
m = Map(center=(51.575, -1.82), zoom=12, basemap=basemaps.CartoDB.Positron)

# Create an output widget for logs
out = widgets.Output(layout={'border': '1px solid black'})

# --- Map Update Function ---
def update_map():
    """A robust function to clear all data layers and redraw from the current state."""
    # It's crucial to remove old layers before adding new ones.
    # We iterate over a copy of the layers list because we are modifying it.
    # The first layer (m.layers[0]) is the basemap, which we don't want to remove.
    layers_to_remove = list(m.layers[1:])
    for layer in layers_to_remove:
        m.remove_layer(layer)
    
    # Get new layers based on the current state of the 'all_facilities' dataframe
    [demand_layer], facility_markers = allocate_demand_and_get_layers(all_facilities, demand_points)
    
    # Add the newly generated layers to the map
    m.add_layer(demand_layer)
    for marker in facility_markers:
        m.add_layer(marker)

# --- Click Handler Function ---
def handle_interaction(**kwargs):
    """This function is called every time the user clicks on the map."""
    global all_facilities
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        lat, lon = coords[0], coords[1]
        
        # Create a new facility and add it to our main dataframe
        new_name = f"Proposed Site {len(all_facilities) - len(existing_facilities) + 1}"
        new_facility = gpd.GeoDataFrame(
            {'name': [new_name]},
            geometry=[Point(lon, lat)], # GeoDataFrame uses (lon, lat)
            crs="EPSG:4326"
        )
        all_facilities = pd.concat([all_facilities, new_facility], ignore_index=True)
        
        # Log the action and update the map
        with out:
            out.clear_output()
            print(f"Adding new facility '{new_name}' at (Lat: {lat:.4f}, Lon: {lon:.4f})")
        update_map()

# --- Reset Button ---
reset_button = widgets.Button(description="Reset Scenario")
def on_reset_button_clicked(b):
    global all_facilities
    with out:
        out.clear_output()
        print("Resetting to initial state.")
    all_facilities = existing_facilities.copy()
    update_map()
reset_button.on_click(on_reset_button_clicked)

# --- Final Assembly ---
# Attach the click handler to the map
m.on_interaction(handle_interaction)

# Draw the initial map state before displaying
update_map()

# Display the final layout
display(widgets.VBox([widgets.HTML("<h3>Click on the map to add a new facility</h3>"), m, reset_button, out]))

### 5. How to Use

1.  Run all the cells above.
2.  The map will display showing the initial state: two existing facilities and the demand points colored according to which of the two is closer.
3.  **Click anywhere on the map.** A new white marker will appear, representing a proposed facility.
4.  The colors of the demand points will instantly update, showing the new catchment areas. You can see how many demand points are now served by your proposed site.
5.  Click the **Reset Scenario** button to remove all proposed sites and return to the initial state.